In [73]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])


import pandas as pd

#criar o navegadorb
nav = webdriver.Chrome(service=servico)

# importar/modificar/visualizar a base de dados
# Importar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

#Modificando a base
tabela_produtos.loc[tabela_produtos["Nome"] == "iphone 12 64gb" , "Nome"] = "Acer nitro 5"
tabela_produtos.loc[tabela_produtos["Nome"] == "rtx 3060" , "Nome"] = "Redmi note 11"

tabela_produtos["Termos banidos"] = "Usado"

tabela_produtos.loc[tabela_produtos["Preço mínimo"] == 3000 , "Preço mínimo"] = "4000"
tabela_produtos.loc[tabela_produtos["Preço mínimo"] == 4000 , "Preço mínimo"] = 1300

tabela_produtos.loc[tabela_produtos["Preço máximo"] == 3500 , "Preço máximo"] = 5500
tabela_produtos.loc[tabela_produtos["Preço máximo"] == 4500 , "Preço máximo"] = 2000

tabela_produtos.loc[2] = ["Acer aspire 5" , "Usado" , "2000" , "3000"]

#Visualizando nova base
display(tabela_produtos)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Usuario\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Acer nitro 5,Usado,4000,5500
1,Redmi note 11,Usado,1300,2000
2,Acer aspire 5,Usado,2000,3000


Definição das funções de busca no google e no buscape

In [74]:
import time

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # entrar no google
    time.sleep(3)
    nav.get("https://www.google.com/")
    time.sleep(5)
    
    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    

    # pesquisar o nome do produto no google
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    # clicar na aba shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar a lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = [] # lista que a função vai me dar como resposta
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        # verificacao do nome - se no nome tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto: # verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue

            
    return lista_ofertas
    
    

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    nav.get("https://www.buscape.com.br/")
    
    # pesquisar pelo produto no buscape
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(10)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Cell_Content__1630r')
    #Cell_Content__1630r
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            continue
    return lista_ofertas

def busca_magazine_luiza(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    nav.get("https://www.magazineluiza.com.br/")
    time.sleep(5)
   
    nav.find_element(By.ID , "input-search").send_keys(produto , Keys.ENTER)
    time.sleep(10)
    lista_resultados = nav.find_elements(By.CLASS_NAME , "sc-fpYaaq oPzXW")
    
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'sc-hKwDye dWfgMa sc-haTkiu dvJtmN').text
            nome = resultado.find_element(By.CLASS_NAME , "price-default").text
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas

Contrução da nossa lista de ofertas encontradas

In [ ]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
        
    lista_ofertas_magazine_luiza = busca_magazine_luiza(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_magazine_luiza:
        tabela_magazine_luiza = pd.DataFrame(lista_ofertas_magazine_luiza, columns=['produto', 'preco', 'link'])
        tabela_ofertas =  tabela_ofertas.append(tabela_magazine_luiza)
    else:
         tabela_magazine_luiza = None

display(tabela_ofertas)

<ipython-input-75-eb2d01a951b1>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)


Exportar a base de ofertas para Excel

In [ ]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)


Enviando o e-mail

In [ ]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'taylannovaes@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()
else:
    print("Nenhum e-mail foi enviado")
nav.quit()  